In [513]:
import requests
import urllib3
import pandas as pd
from datetime import datetime
from datetime import time
from datetime import date

Interesting link: https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde
Strava API forum: https://groups.google.com/g/strava-api

In [514]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
payload = {
    'client_id': "75483",
    'client_secret': 'd7d1e5483201559ddf62c677ac12f6cf7847c5ae',
    'refresh_token': '75994cf8182a3b6a956390b5a4da3eba764f1d85',
    'grant_type': "refresh_token",
    'f': 'json',
}

In [515]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = 5522c2a9953e4dbaa6a5c4da77cf98254b342501



In [517]:
activites_url = "https://www.strava.com/api/v3/athlete/activities"

dataset=[]
pages_count=1

while pages_count>0:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': pages_count}
    my_dataset=requests.get(activites_url, headers=header, params=param).json()
    if my_dataset==[]:
        break
    elif my_dataset!=[]:
        dataset.append(my_dataset)
        pages_count+=1
dataset = [j for i in dataset for j in i]


In [518]:
activities = pd.json_normalize(dataset)

#Create new dataframe with only columns I care about
cols = ['name', 'upload_id', 'type', 'distance', 'moving_time',   
         'average_speed', 'max_speed','total_elevation_gain',
         'start_date_local'
       ]

activities = activities[cols]

#Break date into start time and date
activities['start_date_local']= [datetime.strptime(i,"%Y-%m-%dT%H:%M:%SZ") for i in activities['start_date_local']]

activities['start_time']=[time(hour=i.hour,minute=i.minute,second=i.second) for i in activities['start_date_local']]
activities['start_date_local'] =[date(year=i.year,month=i.month,day=i.day) for i in activities['start_date_local']]
activities.head(10)

,name,upload_id,type,distance,moving_time,average_speed,max_speed,total_elevation_gain,start_date_local,start_time
0,Afternoon Run,6819310648,Run,8061.9,2497,3.229,3.920,109.8,2021-12-22,13:12:30
1,Evening Swim,6812128544,Swim,1200.0,1419,0.846,1.336,0.0,2021-12-20,18:49:18
2,Lunch Ride,6798101336,Ride,20650.0,4972,4.153,9.545,357.3,2021-12-17,12:30:19
3,Afternoon Run,6785614520,Run,7942.9,2554,3.110,4.383,120.3,2021-12-14,16:50:01
4,Afternoon Swim,6785613275,Swim,1300.0,1565,0.831,1.261,0.0,2021-12-13,17:43:20
5,Morning Run,6775126011,Run,9985.1,2939,3.397,4.268,76.8,2021-12-12,09:37:47
6,Afternoon Run,6763110190,Run,9088.0,3043,2.987,3.685,114.1,2021-12-09,13:21:36
7,Lunch Ride,6750142425,Ride,47441.7,10304,4.604,11.381,644.9,2021-12-06,11:15:28
8,Afternoon Swim,6749268660,Swim,1200.0,1391,0.863,2.191,0.0,2021-12-02,17:18:43
9,Afternoon Run,6728577787,Run,5939.6,1878,3.163,4.486,56.0,2021-12-01,17:19:39


In [519]:
activities['type'].value_counts()

Ride              165
Run               111
Swim               78
Hike               49
Workout             2
BackcountrySki      1
Name: type, dtype: int64